## Multi Task Learning(MTL) 기본 모델

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
import os
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

In [ ]:
# 하이퍼파라미터 설정
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# GPU 설정
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\ho_e_\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


## 데이터셋 라벨링
- CustomDataset 클래스에서 이미지 경로를 읽고, **파일명에 포함된 정보**를 바탕으로 라벨을 추출
    - '_'를 기준으로 다섯 번째 값은 질병 정보
    - '_'를 기준으로 여섯 번째 값은 작물 정보
- 작물 번호와 질병 번호를 **라벨 매핑**을 통해 각각 0부터 시작하는 인덱스로 변환
    - 원본 데이터셋의 일부 데이터만 사용하기 때문에 라벨값이 개수 범위를 벗어남 → 매핑 진행

In [ ]:
# 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, root_dir, label_map, transform=None):
        self.root_dir = root_dir
        self.label_map = label_map  # 라벨 맵을 인자로 받음
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        path = self.image_paths[index]
        filename = os.path.basename(path).split('_')

        # 라벨 추출
        disease_label = int(filename[4])  # 다섯 번째 항목이 질병 번호
        crop_label = int(filename[5])     # 여섯 번째 항목이 작물 번호

        # 라벨을 맵핑하여 처리
        crop_label = torch.tensor(self.label_map['crop'][crop_label], dtype=torch.long)  
        disease_label = torch.tensor(self.label_map['disease'][disease_label], dtype=torch.long)

        # 이미지 로드 및 변환
        image = Image.open(path).convert('RGB')
        image = self.transform(image)

        return image, crop_label, disease_label

In [ ]:
# 라벨 맵 정의
label_map = {
    'crop': {1: 0, 2: 1, 3: 2, 6: 3, 9: 4},  # 숫자와 인덱스를 매핑
    'disease': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 11: 7, 12: 8, 16: 9, 17: 10, 18: 11}
}

# 데이터셋 준비
train_dataset = CustomDataset(root_dir='./mtl_dataset/Training', label_map=label_map)
val_dataset = CustomDataset(root_dir='./mtl_dataset/Validation', label_map=label_map)
test_dataset = CustomDataset(root_dir='./mtl_dataset/Test', label_map=label_map)

In [ ]:
# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
# 모델 정의
class MultiTaskModel(nn.Module):
    def __init__(self, backbone, num_crops, num_diseases):
        super(MultiTaskModel, self).__init__()
        self.backbone = backbone  # ResNet-34 backbone
        self.n_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # 마지막 fully connected layer(분류기) 제거
        
        # 태스크 특화 분류기
        self.crop_head = nn.Linear(self.n_features, num_crops)          # 작물 클래스
        self.disease_head = nn.Linear(self.n_features, num_diseases)    # 질병 클래스
    
    def forward(self, x):
        features = self.backbone(x)  # 공유 backbone
        crop_output = self.crop_head(features)  # 작물 분류 출력값
        disease_output = self.disease_head(features)  # 질병 분류 출력값
        return crop_output, disease_output

### 모델 생성
1. 공유 백본
    - `resnet34(weights=ResNet34_Weights.DEFAULT)`
    - 사전학습된 ResNet-34 모델 사용
2. 모델 생성
    - `MultiTaskModel(backbone, num_crops, num_diseases).to(device)`
    - 분류할 클래스 수와 공유 백본을 인수로 모델 생성

In [ ]:
# 모델 생성
num_crops = 5  # 작물 클래스 수
num_diseases = 12  # 질병 클래스 수
backbone = resnet34(weights=ResNet34_Weights.DEFAULT)  # 사전학습된 ResNet-34
model = MultiTaskModel(backbone, num_crops, num_diseases).to(device)

## 모델 평가

In [ ]:
# 모델 평가
model.load_state_dict(torch.load('best_mtl_model.pth', weights_only=True))
model.eval()

correct_crop = 0
correct_disease = 0
total = 0

pred_crop_labels = []
pred_disease_labels = []
true_crop_labels = []
true_disease_labels = []

with torch.no_grad():
    for images, crop_labels, disease_labels in test_loader:
        images = images.to(device, non_blocking=True)
        crop_labels = crop_labels.to(device, non_blocking=True)
        disease_labels = disease_labels.to(device, non_blocking=True)

        crop_outputs, disease_outputs = model(images)
        _, crop_predicted = torch.max(crop_outputs, 1)
        _, disease_predicted = torch.max(disease_outputs, 1)

        total += crop_labels.size(0)
        correct_crop += (crop_predicted == crop_labels).sum().item()
        correct_disease += (disease_predicted == disease_labels).sum().item()

        pred_crop_labels.extend(crop_predicted.cpu().numpy())
        pred_disease_labels.extend(disease_predicted.cpu().numpy())
        true_crop_labels.extend(crop_labels.cpu().numpy())
        true_disease_labels.extend(disease_labels.cpu().numpy())

test_crop_acc = 100 * correct_crop / total
test_disease_acc = 100 * correct_disease / total

print(f"Test Crop Accuracy: {test_crop_acc:.2f}%")
print(f"Test Disease Accuracy: {test_disease_acc:.2f}%")

In [ ]:
# 혼동 행렬 시각화 함수
def plot_confusion_matrix(true_labels, pred_labels, classes, title, save_path=None):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(8, 6))

    # 한글 폰트 설정
    plt.rcParams['font.family'] = 'Malgun Gothic'  # 맑은고딕
    plt.rcParams['axes.unicode_minus'] = False   # 마이너스 기호 깨짐 방지

    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

In [ ]:
# 혼동 행렬 시각화
plot_confusion_matrix(
    true_crop_labels, 
    pred_crop_labels, 
    classes=['고추', '무', '배추', '오이', '파'], 
    title="Crop Classification Confusion Matrix"
)

plot_confusion_matrix(
    true_disease_labels, 
    pred_disease_labels, 
    classes=['정상', '고추탄저병', '고추흰가루병', '무검은무늬병', '무노균병', '배추검음썩음병', '배추노균병', '오이노균병', '오이흰가루병', '파검은무늬병', '파노균병', '파녹병'], 
    title="Disease Classification Confusion Matrix"
)